In [1]:
# USAGE
# python detect_mask_image.py --image images/pic1.jpeg

# import the necessary packages
from keras.applications.mobilenet_v2 import preprocess_input
from keras.utils import img_to_array
from keras.models import load_model
import numpy as np
import argparse
import cv2
import os
import numpy as np


video_file = "Input_Videos/Test_video1.mp4"



# image=r"Input_Videos\Test_video1-opencv\frame0-00-00.03.jpg"
def mask_image(image,counter,model):
    img_size = 124
	# construct the argument parser and parse the arguments
	# ap = argparse.ArgumentParser()
	# ap.add_argument("-i", "--image", required=True,
	# 	help="path to input image")
	# ap.add_argument("-f", "--face", type=str,
	# 	default="face_detector",
	# 	help="path to face detector model directory")
	# ap.add_argument("-m", "--model", type=str,
	# 	default="mask_detector.model",
	# 	help="path to trained face mask detector model")
	# ap.add_argument("-c", "--confidence", type=float, default=0.5,
	# 	help="minimum probability to filter weak detections")
	# args = vars(ap.parse_args())

	# load our serialized face detector model from disk
	
	
    prototxtPath = "face_detector/deploy.prototxt"
    name='' + image
    weightsPath = "face_detector/res10_300x300_ssd_iter_140000.caffemodel"
    net = cv2.dnn.readNet(prototxtPath, weightsPath)
    
    
    # print("[INFO] loading face mask detector model...")
    model = load_model(model)
    image = cv2.imread(image)
    orig = image.copy()
    (h, w) = image.shape[:2]
    
    # construct a blob from the image
    blob = cv2.dnn.blobFromImage(image, 1.0, (300, 300),(104.0, 177.0, 123.0))
    # pass the blob through the network and obtain the face detections
    
    
    # print("[INFO] computing face detections...")
    net.setInput(blob)
    detections = net.forward()
    # loop over the detections
    for i in range(0, detections.shape[2]):
        # print("woo")
        # extract the confidence (i.e., probability) associated with
		# the detection
        confidence = detections[0, 0, i, 2]
        if confidence > 0.15:
            # print("hii")
		# filter out weak detections by ensuring the confidence is
		# greater than the minimum confidence
            box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
            (startX, startY, endX, endY) = box.astype("int")
            
            
            (startX, startY) = (max(0, startX), max(0, startY))
            (endX, endY) = (min(w - 1, endX), min(h - 1, endY))
            
            

			# extract the face ROI, convert it from BGR to RGB channel
			# ordering, resize it to 224x224, and preprocess it
            face = image[startY:endY, startX:endX]
            face = cv2.cvtColor(face, cv2.COLOR_BGR2RGB)
            face = cv2.resize(face,(img_size,img_size))
            face = np.array(face)/255.0
            face = face.reshape(1,img_size,img_size,3)
            
            # pass the face through the model to determine if the face
			# has a mask or not
   
            result = model.predict(face)
            if result>0.5:
                label = "Mask"
            else:
                label = "No Mask"
            
            # determine the class label and color we'll use to draw
			# the bounding box and text
            # label = "Mask" if mask > withoutMask else "No Mask"
            color = (0, 255, 0) if label == "Mask" else (0, 0, 255)
            
			# include the probability in the label
            # label = "{}: {:.2f}%".format(label, max(mask, withoutMask) * 100)
            
            # display the label and bounding box rectangle on the output
			# frame
   
   
            cv2.putText(image, label, (startX, startY - 10),cv2.FONT_HERSHEY_SIMPLEX, 0.45, color, 2)
            cv2.rectangle(image, (startX, startY), (endX, endY), color, 2)
     
     # show the output image   
     
    filename, _ = os.path.splitext(video_file)
    filename += "-forvid"
    # make a folder by the name of the video file
    if not os.path.isdir(filename):
        os.mkdir(filename)    
    cv2.imwrite(os.path.join(filename,f"output{counter:04d}.jpg"), image) 
    # cv2.imshow("Output",image)
    # counter += 1
    
    cv2.waitKey(0)
	
# if __name__ == "__main__":
# mask_image(image)

In [2]:
from datetime import timedelta
import cv2
import numpy as np
import os

# i.e if video of duration 30 seconds, saves 10 frame per second = 300 frames saved in total
SAVING_FRAMES_PER_SECOND = 30

def format_timedelta(td):
    """Utility function to format timedelta objects in a cool way (e.g 00:00:20.05) 
    omitting microseconds and retaining milliseconds"""
    result = str(td)
    try:
        result, ms = result.split(".")
    except ValueError:
        return (result + ".00").replace(":", "-")
    ms = int(ms)
    ms = round(ms / 1e4)
    return f"{result}.{ms:02}".replace(":", "-")


def get_saving_frames_durations(cap, saving_fps):
    """A function that returns the list of durations where to save the frames"""
    s = []
    # get the clip duration by dividing number of frames by the number of frames per second
    clip_duration = cap.get(cv2.CAP_PROP_FRAME_COUNT) / cap.get(cv2.CAP_PROP_FPS)
    # use np.arange() to make floating-point steps
    for i in np.arange(0, clip_duration, 1 / saving_fps):
        s.append(i)
    return s


def main(video_file):
    filename, _ = os.path.splitext(video_file)
    filename += "-opencv"
    # make a folder by the name of the video file
    if not os.path.isdir(filename):
        os.mkdir(filename)
    # read the video file    
    cap = cv2.VideoCapture(video_file)
    # get the FPS of the video
    fps = cap.get(cv2.CAP_PROP_FPS)
    # if the SAVING_FRAMES_PER_SECOND is above video FPS, then set it to FPS (as maximum)
    saving_frames_per_second = min(fps, SAVING_FRAMES_PER_SECOND)
    # get the list of duration spots to save
    saving_frames_durations = get_saving_frames_durations(cap, saving_frames_per_second)
    # start the loop
    count = 0
    while True:
        is_read, frame = cap.read()
        if not is_read:
            # break out of the loop if there are no frames to read
            break
        # get the duration by dividing the frame count by the FPS
        frame_duration = count / fps
        try:
            # get the earliest duration to save
            closest_duration = saving_frames_durations[0]
        except IndexError:
            # the list is empty, all duration frames were saved
            break
        if frame_duration >= closest_duration:
            # if closest duration is less than or equals the frame duration, 
            # then save the frame
            frame_duration_formatted = format_timedelta(timedelta(seconds=frame_duration))
            cv2.imwrite(os.path.join(filename, f"frame{frame_duration_formatted}.jpg"), frame) 
            
            
            
            # drop the duration spot from the list, since this duration spot is already saved
            try:
                saving_frames_durations.pop(0)
            except IndexError:
                pass
        # increment the frame count
        count += 1



# if __name__ == "__main__":
#     # import sys
#     video_file = "Input_Videos/Test_video2.mp4"
#     main(video_file)

main(video_file)

In [3]:



def process_files_in_folder(folder_path):
    model="mask_detector2.model"
    counter=0
    for root, dirs, files in os.walk(folder_path):
        for file in files:
            file_path = os.path.join(root, file)
            mask_image(file_path,counter,model)
            counter+=1
folder_path = "Input_Videos/Test_video1-opencv"
process_files_in_folder(folder_path)

1/1 [==============================] - 0s 115ms/step


KeyboardInterrupt: 

In [ ]:
# Define the video parameters
folder_path = "Input_Videos/Test_video1-forvid"
fps = 30.0
frame_size = (640, 360)
fourcc = cv2.VideoWriter_fourcc(*"mp4v")
# Create a VideoWriter object
out = cv2.VideoWriter("output.mp4", fourcc, fps, frame_size)
# Read the images and write them to the video
for root, dirs, files in os.walk(folder_path):
        for file in files:
            file_path = os.path.join(root, file)
            
            img = cv2.imread(file_path)
            out.write(img)

    
    
# Release the VideoWriter object
out.release()